# Punto 1

In [3]:
import json
import pandas as pd

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
import ssl

try:
     _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')#%% md

## Reading Files

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
italian_path = './datasets/italian/italianV1.json'
spanish_path = './datasets/spanish/spanishV1.json'
english_path = './datasets/english/englishV1.json'


In [5]:
file = open(english_path, 'r')
data = []
for line in file:
    data.append(json.loads(line))

english_raw = pd.json_normalize(data)
print(english_raw.shape)
english_raw

(817046, 4)


,id,publication_date,source,text
0,1295929115770593287,1.597809e+09,twitter,Info Source:
1,1296738518216011777,1.598002e+09,twitter,#PostponeJEE_NEETSept #ProtestAgainstExamsInCO...
2,1252450676015198210,1.587442e+09,twitter,Coronavirus-spreader Chris Cuomo got a lecture...
3,1380684968880406528,1.618016e+09,twitter,Any military member that refuses to get vaccin...
4,1368958702150156290,1.615220e+09,twitter,#Covid19 is staying around for a while. your ...
...,...,...,...,...
817041,1270423398120935426,1.591728e+09,twitter,"Children receiving free school meals, from sin..."
817042,1283969389117792258,1.594957e+09,twitter,L.A. County reports highest number of coronavi...
817043,1309834472053903360,1.601124e+09,twitter,Progress ?Covid19 Political Development
817044,1279062456468492289,1.593787e+09,twitter,@WeeklyEconPod @AyeishaTS @JCWI_UK @ZoeJardini...


In [29]:
'''
Turn text to Numpy Array
'''
texts_column = english_raw.loc[:,'text']
raw_texts = texts_column.values

In [30]:
'''
Crop for testing
'''
#raw_texts = raw_texts[:1000]

'\nCrop for testing\n'

In [31]:
stemmer = SnowballStemmer("english")
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))

    return result

In [32]:
processed_docs = []

for doc in raw_texts:
    processed_docs.append(preprocess(doc))

In [33]:
'''
Preview 'processed_docs'
'''
print(processed_docs[:10])

[['info', 'sourc'], ['work'], ['coronavirus', 'spreader', 'chris', 'cuomo', 'lectur', 'break', 'quarantin', 'upset'], ['militari', 'member', 'refus', 'vaccin', 'baffl', 'shit', 'pump'], ['covid', 'stay', 'busi', 'covidsaf'], ['lie', 'lie'], ['gregmannarino', 'deborah', 'birxwhit', 'hous', 'expert'], ['kayleigh', 'mcenani', 'scienc', 'stand', 'reopen', 'school'], ['amaz', 'effort', 'guy', 'donat'], ['sulaiodus', 'say', 'suspend', 'coronavirus', 'media']]


In [34]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [35]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [36]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [37]:
# LDA multicore
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                   num_topics = 8,
                                   id2word = dictionary,
                                   passes = 10,
                                   workers = 2)

In [38]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.022*"vaccin" + 0.013*"patient" + 0.012*"hous" + 0.012*"studi" + 0.010*"hospit" + 0.010*"research" + 0.009*"expert" + 0.009*"say" + 0.009*"doctor" + 0.008*"post"


Topic: 1 
Words: 0.028*"mask" + 0.019*"pandem" + 0.016*"china" + 0.014*"face" + 0.014*"wear" + 0.012*"govern" + 0.009*"market" + 0.008*"sign" + 0.007*"world" + 0.007*"wuhan"


Topic: 2 
Words: 0.068*"peopl" + 0.039*"like" + 0.019*"look" + 0.018*"think" + 0.013*"right" + 0.011*"kill" + 0.011*"fuck" + 0.011*"get" + 0.009*"long" + 0.009*"go"


Topic: 3 
Words: 0.040*"vaccin" + 0.020*"health" + 0.017*"help" + 0.012*"home" + 0.012*"pandem" + 0.011*"stay" + 0.011*"busi" + 0.010*"work" + 0.010*"worker" + 0.009*"care"


Topic: 4 
Words: 0.067*"case" + 0.046*"test" + 0.041*"death" + 0.028*"report" + 0.022*"posit" + 0.020*"updat" + 0.014*"number" + 0.013*"state" + 0.010*"news" + 0.010*"infect"


Topic: 5 
Words: 0.029*"know" + 0.020*"good" + 0.019*"come" + 0.018*"year" + 0.016*"go" + 0.011*"time" + 0.010*"hope" + 0.0